In [1]:
%load_ext autoreload
%autoreload 2

import logging

import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import defi.dao_cross.dao_cross as ddcrdacr
import defi.dao_cross.order as ddacrord

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)
/tmp/ipykernel_1382/3057784219.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
hdbg.init_logger(verbosity=logging.DEBUG)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-26f4ad08-6f31-4966-8385-11e470abe62a.json'
DEBUG Effective logging level=10
DEBUG Shut up 1 modules: asyncio
DEBUG > (cd . && cd "$(git rev-parse --show-toplevel)/.." && (git rev-parse --is-inside-work-tree | grep -q true)) 2>&1
DEBUG > (git rev-parse --show-toplevel) 2>&1
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.3' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
DEBUG > (git branch --show-current) 2>&1
DEBUG > (git rev-parse --short HEAD) 2>&1
DEBUG > (git log --date=local --oneline --graph --date-order --decorate --pretty=format:'%h %<(8)%aN%  %<(65)%s (%>(14)%ar) %ad %<(10)%d' -3)

# Order class

In [4]:
buy_order1 = ddacrord.Order("ETH", "BTC", "buy", 10, 0.1, 1, 1)
buy_order2 = ddacrord.Order("ETH", "BTC", "buy", 9, 0.15, 2, 2)
buy_order3 = ddacrord.Order("ETH", "BTC", "buy", 10, 0.15, 3, 3)
buy_orders = [buy_order1, buy_order2, buy_order3]
display(buy_orders)

[base_token=ETH quote_token=BTC action=buy quantity=10 limit_price=0.1 timestamp=2023-03-21 19:48:07.241166+00:00 deposit_address=1,
 base_token=ETH quote_token=BTC action=buy quantity=9 limit_price=0.15 timestamp=2023-03-21 19:48:07.242983+00:00 deposit_address=2,
 base_token=ETH quote_token=BTC action=buy quantity=10 limit_price=0.15 timestamp=2023-03-21 19:48:07.243247+00:00 deposit_address=3]

In [5]:
sell_order1 = ddacrord.Order("ETH", "BTC", "sell", 11, 0.1, -1, -1)
sell_order2 = ddacrord.Order("ETH", "BTC", "sell", 10, 0.15, -2, -2)
sell_order3 = ddacrord.Order("ETH", "BTC", "sell", 9, 0.15, -3, -3)
sell_orders = [sell_order1, sell_order2, sell_order3]
display(sell_orders)

[base_token=ETH quote_token=BTC action=sell quantity=11 limit_price=0.1 timestamp=2023-03-21 19:48:07.353962+00:00 deposit_address=-1,
 base_token=ETH quote_token=BTC action=sell quantity=10 limit_price=0.15 timestamp=2023-03-21 19:48:07.354294+00:00 deposit_address=-2,
 base_token=ETH quote_token=BTC action=sell quantity=9 limit_price=0.15 timestamp=2023-03-21 19:48:07.354509+00:00 deposit_address=-3]

In [7]:
ddacrord.get_random_order()

base_token=ETH quote_token=BTC action=buy quantity=8 limit_price=inf timestamp=2023-03-21 19:48:07.526487+00:00 deposit_address=3

# Order matching

## Example 1

In [9]:
clearing_price = 0.14
transfers_df = ddcrdacr.match_orders(buy_orders + sell_orders, clearing_price)

DEBUG Order not eligible for matching due to limit
DEBUG Order not eligible for matching due to limit
DEBUG Order not eligible for matching due to limit
DEBUG buy_heap=[base_token=ETH quote_token=BTC action=buy quantity=10 limit_price=0.15 timestamp=2023-03-21 19:48:07.243247+00:00 deposit_address=3, base_token=ETH quote_token=BTC action=buy quantity=9 limit_price=0.15 timestamp=2023-03-21 19:48:07.242983+00:00 deposit_address=2]
DEBUG sell_heap=[base_token=ETH quote_token=BTC action=sell quantity=11 limit_price=0.1 timestamp=2023-03-21 19:48:07.353962+00:00 deposit_address=-1]


In [10]:
eth_transfers = transfers_df[transfers_df["token"] == "ETH"]
display(eth_transfers)

,token,amount,from,to
0,ETH,9,-1,2
2,ETH,2,-1,3


In [11]:
# Transferred ETH (in ETH).
eth_transfers["amount"].sum()

11

In [12]:
btc_transfers = transfers_df[transfers_df["token"] == "BTC"]
display(btc_transfers)

,token,amount,from,to
1,BTC,1.26,2,-1
3,BTC,0.28,3,-1


In [13]:
# Transferred BTC (in BTC).
btc_transfers["amount"].sum()

1.5400000000000003

In [14]:
# BTC transferred divided by the ETH/BTC price (quoted in BTC per ETH)
# returns amount of transferred ETH.
btc_transfers["amount"].sum() / clearing_price

11.0

## Example 2

In [15]:
orders = []
for idx in range(0, 10):
    order = ddacrord.get_random_order(idx)
    orders.append(order)

In [16]:
orders

[base_token=ETH quote_token=BTC action=sell quantity=6 limit_price=-inf timestamp=2023-03-21 19:48:08.314433+00:00 deposit_address=-3,
 base_token=ETH quote_token=BTC action=buy quantity=9 limit_price=inf timestamp=2023-03-21 19:48:08.314702+00:00 deposit_address=2,
 base_token=ETH quote_token=BTC action=buy quantity=9 limit_price=inf timestamp=2023-03-21 19:48:08.314846+00:00 deposit_address=0,
 base_token=ETH quote_token=BTC action=sell quantity=4 limit_price=-inf timestamp=2023-03-21 19:48:08.314971+00:00 deposit_address=-3,
 base_token=ETH quote_token=BTC action=sell quantity=8 limit_price=-inf timestamp=2023-03-21 19:48:08.315092+00:00 deposit_address=-2,
 base_token=ETH quote_token=BTC action=buy quantity=7 limit_price=inf timestamp=2023-03-21 19:48:08.315212+00:00 deposit_address=3,
 base_token=ETH quote_token=BTC action=sell quantity=4 limit_price=-inf timestamp=2023-03-21 19:48:08.315348+00:00 deposit_address=-1,
 base_token=ETH quote_token=BTC action=buy quantity=7 limit_pric

In [17]:
transfers_df_2 = ddcrdacr.match_orders(orders, 0.1)
display(transfers_df_2)

DEBUG buy_heap=[base_token=ETH quote_token=BTC action=buy quantity=6 limit_price=inf timestamp=2023-03-21 19:48:08.315701+00:00 deposit_address=3, base_token=ETH quote_token=BTC action=buy quantity=7 limit_price=inf timestamp=2023-03-21 19:48:08.315472+00:00 deposit_address=0, base_token=ETH quote_token=BTC action=buy quantity=9 limit_price=inf timestamp=2023-03-21 19:48:08.314846+00:00 deposit_address=0, base_token=ETH quote_token=BTC action=buy quantity=9 limit_price=inf timestamp=2023-03-21 19:48:08.314702+00:00 deposit_address=2, base_token=ETH quote_token=BTC action=buy quantity=7 limit_price=inf timestamp=2023-03-21 19:48:08.315212+00:00 deposit_address=3]
DEBUG sell_heap=[base_token=ETH quote_token=BTC action=sell quantity=2 limit_price=-inf timestamp=2023-03-21 19:48:08.315587+00:00 deposit_address=-2, base_token=ETH quote_token=BTC action=sell quantity=4 limit_price=-inf timestamp=2023-03-21 19:48:08.315348+00:00 deposit_address=-1, base_token=ETH quote_token=BTC action=sell q

,token,amount,from,to
0,ETH,7,-2,3
1,BTC,0.7,3,-2
2,ETH,1,-2,2
3,BTC,0.1,2,-2
4,ETH,6,-3,2
5,BTC,0.6,2,-3
6,ETH,2,-3,2
7,BTC,0.2,2,-3
8,ETH,2,-3,0
9,BTC,0.2,0,-3
